# DSI-24 EEG Triggering: Concepts and Best Practices

By the end of this notebook, you should understand the concepts of tonic vs. phasic triggers, immediate vs. on-flip timing in PsychoPy, and best practices for establishing a robust serial communication between your experiment software and the DSI-24 amplifier.

---

## Table of Contents
1. [Introduction](#introduction)  
2. [Background on DSI-24 and Triggering](#background-on-dsi-24-and-triggering)  
   1. [Hardware and Serial Communication](#hardware-and-serial-communication)  
3. [Tonic vs. Phasic Triggers](#tonic-vs-phasic-triggers)  
   1. [Tonic (Sustained) Triggers](#tonic-sustained-triggers)  
   2. [Phasic (Brief) Triggers](#phasic-brief-triggers)  
   3. [Choosing the Right Approach](#choosing-the-right-approach)  
4. [Immediate vs. On-Flip Timing](#immediate-vs-on-flip-timing)  
5. [Opening and Managing the Serial Connection](#opening-and-managing-the-serial-connection)  
6. [Additional Best Practices](#additional-best-practices)  
7. [Conclusion](#conclusion)  

---

<a name="introduction"></a>
## 1. Introduction

In many EEG experiments, sending **event markers** (also called *triggers*) from your behavioral task software (e.g., [PsychoPy](https://www.psychopy.org/)) to the **DSI-24 wireless EEG headset** is crucial for synchronizing recorded brain signals with specific paradigm events (e.g., stimulus onsets, condition changes, participant responses). 

This tutorial introduces you to:
- The difference between **tonic** (sustained) and **phasic** (brief) triggers.
- How to handle **immediate** vs. **on-flip** timing when drawing frames in PsychoPy.
- Best practices for opening and managing a reliable **serial connection** to ensure high-quality EEG event alignment.

> **Note**: For Python code snippets that demonstrate how to implement these concepts in practice, please refer to the accompanying [*DSI-24 EEG Triggering: Code Snippets & Quick Reference*](#) document.

---

<a name="background-on-dsi-24-and-triggering"></a>
## 2. Background on DSI-24 and Triggering

The **DSI-24** system is a **wireless EEG headset** capable of receiving numeric trigger codes through a **serial port** or other specialized input interface. Each numeric code is stored alongside the EEG data, allowing you to precisely mark the time point at which specific events occurred during data acquisition.

<a name="hardware-and-serial-communication"></a>
### 2.1 Hardware and Serial Communication

- **Physical Connection**  
  Typically, you connect your computer (running experiment software like PsychoPy) to the DSI-24’s data acquisition software (e.g., **DSI-Streamer**) through a **USB-to-serial adapter** or a dedicated trigger interface. Ensure the hardware is properly configured and recognized by your operating system.

- **Finding Your COM Port**  
  *Windows*  
  1. Press **Win+X** and select “Device Manager”.  
  2. Expand **“Ports (COM & LPT)”** to locate the **USB Serial Port (COMX)**.  

  *Linux*  
  1. Plug in your device, then open a terminal.  
  2. Run:  
     ```bash
     dmesg | grep tty
     ```  
     Look for something like `/dev/ttyUSB0`.

  *macOS*  
  1. Open **Terminal**.  
  2. Run:  
     ```bash
     ls /dev/tty.*
     ls /dev/cu.*
     ```  
     You should see entries referencing `usbserial` or similar.

<p align="center">
  <img src="https://via.placeholder.com/400x150?text=COM+Port+Diagram" alt="Example: COM Port Diagram" width="50%"/>
</p>

Identifying your device’s port name is essential before configuring the serial connection in Python or PsychoPy.

For examples of initializing and resetting your trigger line (i.e., setting it to 0 at the start), see **Section 1** of the [Code Snippets & Quick Reference](#) document.

---

<a name="tonic-vs-phasic-triggers"></a>
## 3. Tonic vs. Phasic Triggers

Event markers, or triggers, help align your EEG data with specific events in your experiment. Two commonly used trigger styles are **tonic** and **phasic**.

<a name="tonic-sustained-triggers"></a>
### 3.1 Tonic (Sustained) Triggers
- **Definition**  
  A **tonic** trigger is set to a nonzero value at the start of a block or event and remains at that value until it is explicitly reset to zero.
  
- **When to Use**  
  - **Long Blocks/States**: Great for multi-minute intervals or tasks where you want a continuous marker (e.g., a 5-minute baseline).  
  - **Neurofeedback/Closed-Loop**: Useful for real-time tracking and feedback, as a constant marker can signal the current state to the software.

- **Pros/Cons**  
  - **Pros**:  
    - Easy to visualize in raw data (consistent marker).  
    - Simplifies marking extended durations.  
  - **Cons**:  
    - Requires manual reset to zero at the end.  
    - Less precise for capturing short events inside a block.

For detailed examples of sending a **tonic trigger** or resetting it to zero, refer to:
- **Section 3.1** (_Immediate Tonic Trigger_) in the [Code Snippets & Quick Reference](#)  
- **Section 3.2** (_Reset Tonic Trigger_)  
- **Section 3.3** (_On-Flip Tonic Trigger_, if you need to synchronize with screen flips)

<a name="phasic-brief-triggers"></a>
### 3.2 Phasic (Brief) Triggers
- **Definition**  
  A **phasic** trigger is a short pulse—often 10–50 ms—turned on at a specific time (e.g., stimulus onset) and turned off immediately afterward.

- **When to Use**  
  - **Discrete Stimulus Onsets**: Common in ERP studies (e.g., P300 experiments).  
  - **Rapid-Fire Events**: Each short pulse marks distinct events without overlap.

- **Pros/Cons**  
  - **Pros**:  
    - High temporal precision.  
    - Eliminates overlap between consecutive events.  
  - **Cons**:  
    - Not suitable for extended conditions (pulse is too short to indicate state).  
    - Must ensure the hardware can reliably detect short pulses (minimum pulse width).

For code examples of **phasic triggers** (both **immediate** and **on-flip**), see:
- **Section 2.1** (_Immediate Phasic Trigger_)  
- **Section 2.2** (_On-Flip Phasic Trigger_)

<a name="choosing-the-right-approach"></a>
### 3.3 Choosing the Right Approach
- **Experiment Duration**  
  - **Tonic**: Suitable for multi-second/minute intervals where you need a sustained marker.  
  - **Phasic**: Best for short, discrete events that require precise timing.  
- **Analysis Goals**  
  - **ERP/Time-Locked Studies**: Typically prefer **phasic** triggers.  
  - **Block-Based/Neurofeedback**: Typically prefer **tonic** triggers.  
- **Hybrid Designs**  
  - Many experiments combine a tonic marker to indicate large blocks and phasic markers for precise sub-events.

See **Section 4** of the [Code Snippets & Quick Reference](#) for examples of combining both trigger styles within the same experiment.

---

<a name="immediate-vs-on-flip-timing"></a>
## 4. Immediate vs. On-Flip Timing

In PsychoPy (and other similar frameworks), you also need to decide **when** to send the trigger relative to the screen refresh:

1. **Immediate**  
   - Sends the trigger at the exact moment the code line executes.  
   - **Use Case**: Events not tied to a screen refresh, such as a key press or a logic-based condition.

2. **On-Flip**  
   - Schedules the trigger for the **next screen refresh** (`win.flip()` in PsychoPy).  
   - **Use Case**: Visual stimulus presentation requiring frame-level synchronization (e.g., ERP experiments).

> **Example**: For an ERP study measuring response latency, using **on-flip** triggers can ensure the trigger is perfectly aligned with the start of the stimulus display. Conversely, logging a participant’s button press can be done *immediately* since it’s not dependent on display refresh timing.

For specific functions that demonstrate **on-flip** triggers (either phasic or tonic), see **Section 2.2** and **Section 3.3** of the [Code Snippets & Quick Reference](#) document.

---

<a name="opening-and-managing-the-serial-connection"></a>
## 5. Opening and Managing the Serial Connection

Below are important considerations for opening and configuring your serial connection. For an actual example of how to open the port, set the baud rate, and reset the trigger line to 0, please see **Section 1** of the [Code Snippets & Quick Reference](#).

1. **Identify the COM Port**:  
   Find the USB-to-serial port name (e.g., `COM3`, `/dev/ttyUSB0`).

2. **Initialize and Reset**:  
   It is a best practice to **open** the serial port early in your script and **reset** the trigger line to `0`, ensuring a clean starting state.

3. **Confirm Specs**:  
   Some EEG amplifiers require a minimum pulse width or specific baud rate. Check your **DSI-24** documentation to confirm the recommended serial settings.

> **Tip**: If using PsychoPy, place serial initialization in the “Begin Experiment” tab or at the start of your script to ensure it’s ready by the time you need to send triggers.

---

## 6. Additional Best Practices

1. **Consistent Coding Scheme**  
   - Use a clear, documented mapping of triggers (e.g., `START_TASK = 10`, `GO_ONSET = 1`) so you can easily interpret them later.

2. **Resetting Tonic Triggers**  
   - If using tonic triggers, remember to **reset** back to `0` after the condition ends. Forgetting to do so can cause confusion in downstream analyses.

3. **Timing Requirements**  
   - Check the **minimum pulse width** for your DSI-24 amplifier. A 10 ms pulse is often safe, but confirm hardware specifications.

4. **Logging and Verification**  
   - Always log triggers in your console or a data file. This helps with debugging and ensures triggers are being sent when you expect.

5. **Hybrid Approach**  
   - Combine a tonic marker for major blocks with **phasic** pulses for sub-events or stimulus onsets if your design requires both high-level states and precise timing.

6. **Performance and Scalability**  
   - For larger experiments, ensure that your serial write operations are efficient. If you anticipate very frequent triggers, test for potential bottlenecks or buffering delays.

---

<a name="conclusion"></a>
## 7. Conclusion

Accurate event marking is essential for meaningful EEG data analysis. By understanding **tonic vs. phasic triggers** and using **immediate vs. on-flip** timing appropriately, you can ensure that your DSI-24 EEG data aligns precisely with the experimental events that matter most.

**Key Takeaways**:
- **Tonic vs. Phasic**: Choose based on block duration and desired timing precision.  
- **Immediate vs. On-Flip**: Immediate is best for logic-driven triggers; on-flip is for frame-locked stimuli.  
- **Reliable Serial Connection**: Set up your COM port early, confirm baud rates, and reset triggers to 0 for a clean start.

For further details on actual code implementations, see:
- [*DSI-24 EEG Triggering: Code Snippets & Quick Reference*](#)  
  - **Section 1**: Serial setup and resetting trigger line  
  - **Sections 2 & 3**: Phasic and Tonic trigger functions (immediate/on-flip)  
  - **Section 4**: Combining Tonic & Phasic triggers  

- [*DSI24_GoNoGoDemo_PsychoPy_TriggerImplementation.ipynb*](#) for a more complete example demonstrating a Go/No-Go experimental workflow.

> **Note**: Make sure the dataset or any data you use do not contain sensitive or proprietary information. Always follow your organization’s guidelines for data handling and code documentation.

---

### References & Related Resources

- **PsychoPy** Documentation: [https://www.psychopy.org/](https://www.psychopy.org/)  
- **DSI-24** Documentation: [Wearable Sensing Official Site](https://wearablesensing.com/)  



